In [2]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import psycopg2
from kiwipiepy import Kiwi
import plotly.express as px

# 데이터베이스 연결
db_connection = psycopg2.connect(
    host="localhost",
    dbname="final_project",
    user="postgres",
    password="sesac0916",
    port=5432
)

cursor = db_connection.cursor()
# SELECT 쿼리 실행
cursor.execute("SELECT * FROM final_project.news_information;")
# 결과 가져오기
result = cursor.fetchall()
# 연결 종료
cursor.close()
db_connection.close()

# 데이터프레임으로 불러오기
df_news = pd.DataFrame(result, columns=['id', 'create_dt', 'media_company', 'news_genre', 'news_image_path', 'news_title', 'news_origin', 'news_url', 'news_summary'])

# 'create_dt' 열을 datetime 형식으로 변환
df_news['create_dt'] = pd.to_datetime(df_news['create_dt'])

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# Dash 앱 생성
app = Dash(__name__)

# 뉴스 장르와 날짜의 유니크 값들을 추출
unique_genres = df_news['news_genre'].unique()

app.layout = html.Div([
    dcc.Graph(id="graph"),
    html.P("뉴스 장르:"),
    dcc.Dropdown(id='genres',
        options=[{'label': genre, 'value': genre} for genre in unique_genres],
        value='',  # or value=unique_genres[0] to set the first genre as default
        clearable=False
    ),
])

@app.callback(
    Output("graph", "figure"), 
    Input("genres", "value"))
def generate_chart(selected_genre):
    if selected_genre == '':
        df_selected_data = df_news
    else:
        df_selected_data = df_news[df_news['news_genre'] == selected_genre]

    # Kiwi 형태소 분석 적용
    words = kiwi.tokenize(' '.join(df_selected_data['news_title']))
    words = [token.form for token in words if len(token.form) >= 2 and token.tag in ['NNG', 'NNP', 'NNB', 'NR', 'NP']]
    word_counts = pd.Series(words).value_counts().reset_index(name='n')
    
    # Select top 5 words for the pie chart
    top_words = word_counts.head(5)

    fig = px.pie(top_words, values='n', names='index', hole=.3, title=f"{selected_genre} 장르의 상위 5개 단어 빈도수")
    
    return fig

app.run_server(debug=True, port=8058)
